In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [3]:
train_df = pd.read_csv("../input/wine_red_train.csv")
test_df = pd.read_csv("../input/wine_red_test.csv")

In [4]:
train_df.head()

,fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,quality
0,11.6,0.580,0.66,2.20,0.074,10.0,47.0,1.00080,3.25,0.57,9.0,3
1,10.4,0.610,0.49,2.10,0.200,5.0,16.0,0.99940,3.16,0.63,8.4,3
2,7.4,1.185,0.00,4.25,0.097,5.0,14.0,0.99660,3.63,0.54,10.7,3
3,10.4,0.440,0.42,1.50,0.145,34.0,48.0,0.99832,3.38,0.86,9.9,3
4,7.6,1.580,0.00,2.10,0.137,5.0,9.0,0.99476,3.50,0.40,10.9,3


In [5]:
test_df.head()

,fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,quality
0,8.3,1.020,0.02,3.4,0.084,6.0,11,0.99892,3.48,0.49,11.0,3
1,8.3,0.675,0.26,2.1,0.084,11.0,43,0.99760,3.31,0.53,9.2,4
2,4.6,0.520,0.15,2.1,0.054,8.0,65,0.99340,3.90,0.56,13.1,4
3,8.8,0.955,0.05,1.8,0.075,5.0,19,0.99616,3.30,0.44,9.6,4
4,9.2,0.520,1.00,3.4,0.610,32.0,69,0.99960,2.74,2.00,9.4,4


In [8]:
x_train = train_df.iloc[:, 0:11]
y_train = train_df[["quality"]]
x_test = test_df.iloc[:, 0:11]
y_test = test_df[["quality"]]

In [9]:
x_train.head()

,fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol
0,11.6,0.580,0.66,2.20,0.074,10.0,47.0,1.00080,3.25,0.57,9.0
1,10.4,0.610,0.49,2.10,0.200,5.0,16.0,0.99940,3.16,0.63,8.4
2,7.4,1.185,0.00,4.25,0.097,5.0,14.0,0.99660,3.63,0.54,10.7
3,10.4,0.440,0.42,1.50,0.145,34.0,48.0,0.99832,3.38,0.86,9.9
4,7.6,1.580,0.00,2.10,0.137,5.0,9.0,0.99476,3.50,0.40,10.9


In [10]:
y_train.head()

,quality
0,3
1,3
2,3
3,3
4,3


In [20]:
y_train["quality"].unique()

array([3, 4, 5, 6, 7, 8])

In [21]:
y_train = y_train - 3
y_test = y_test - 3

In [22]:
y_train["quality"].unique()

array([0, 1, 2, 3, 4, 5])

In [11]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.fit_transform(x_test)

In [13]:
x_train

array([[ 1.88721973,  0.29099094,  2.00427189, ..., -0.40461395,
        -0.53183734, -1.33181487],
       [ 1.19943723,  0.45896685,  1.13014764, ..., -0.98860539,
        -0.1647803 , -1.89059659],
       [-0.52001901,  3.67850521, -1.38938696, ...,  2.06112771,
        -0.71536586,  0.2514    ],
       ...,
       [-1.60900797, -0.21293681, -1.23512974, ...,  1.21758451,
         0.997567  ,  3.32469946],
       [-0.63464943, -1.10880835,  0.30744247, ..., -0.53438982,
         2.71049986, -0.40051201],
       [-0.52001901, -0.94083243,  0.15318525, ..., -0.46950189,
         0.26345292,  0.90331201]])

In [26]:
x = tf.placeholder(tf.float32, [None, 11])

W1 = tf.Variable(tf.random_normal([11,9], mean=0.0, stddev=0.5))
b1 = tf.Variable(tf.random_normal([9], mean=0.0, stddev=0.1))
y1 = tf.matmul(x, W1) + b1
tf.get_variable_scope().reuse_variables()

W2 = tf.Variable(tf.random_normal([9,6], mean=0.0, stddev=0.5))
b2 = tf.Variable(tf.random_normal([6], mean=0.0, stddev=0.1))
y2 = tf.nn.softmax(tf.matmul(y1, W2) + b2)
tf.get_variable_scope().reuse_variables()

y = tf.placeholder(tf.int64, [None, 1])
y_ = tf.one_hot(indices = y, depth = 6)

In [27]:
global_step = tf.Variable(0, trainable=False)
starter_learning_rate = 0.3
learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                           1000, 0.95, staircase=True)
tf.get_variable_scope().reuse_variables()

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y_, logits = y2))
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

sess = tf.Session()
init = tf.initialize_all_variables()
sess.run(init)

In [28]:
num_epochs = 500
for i in range(num_epochs):
    sess.run(optimizer, feed_dict = {x:x_train, y:y_train})

pred = sess.run(y2, feed_dict = {x: x_test})
# pred = pred.round(0)
pred_d = tf.argmax(pred,1)

print(confusion_matrix(y_test, sess.run(pred_d)), accuracy_score(y_test, sess.run(pred_d)))

[[ 0  0  1  0  0  0]
 [ 0  0  4  1  0  0]
 [ 0  0 48 19  1  0]
 [ 0  0 16 42  6  0]
 [ 0  0  1 12  7  0]
 [ 0  0  0  2  0  0]] 0.60625
